# Machine Learning

This notebook searchs to use machine learning techniques to model the studied data in the exploratory_analysis.ipynb notebook. First we install and import the required libraries:

In [ ]:
!pip install imblearn
!pip install xgboost

In [103]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn import tree

## Feature Engineering

First of all, the data collected from the satellite images (See the satellite processing folder) is added to the training set:

In [104]:
df = pd.read_csv('../data/Train.csv')
df_encoded = pd.read_csv('../data/train_encoded.csv')
df = pd.concat([df,df_encoded.ndvi,df_encoded.evi,df_encoded.ndwi,df_encoded.gndvi,df_encoded.savi,df_encoded.msavi],axis=1)

About one thousand farms don't have satellite data associted so all they have for the new data are missing values. These records are dropped:

In [105]:
df.dropna(inplace=True)

Just a short multivariate analysis to see what is going on:

In [106]:
df_satellite = pd.concat([df.ndvi,df.ndwi,df.gndvi,df.savi,df.msavi,df.category],axis=1)
df_satellite_gb = df_satellite.groupby('category')
df_satellite_gb.agg(['mean','std'])

ndvi                 ndwi               gndvi            \
              mean       std       mean       std      mean       std   
category                                                                
Diseased  0.568699  1.460792  10.042921  2.677730  0.254835  0.694027   
Healthy   0.682831  1.637722  10.215223  2.771421  0.295578  0.899111   
Pests     0.725410  1.800500  10.218539  2.646600  0.257164  0.553859   
Stressed  0.912816  2.261620  10.440244  2.919329  0.362572  1.271307   

              savi                  msavi               
              mean       std         mean          std  
category                                                
Diseased  0.839677  2.176318  3878.101041  1390.030283  
Healthy   1.015285  2.448178  3845.658390  1505.893835  
Pests     1.085837  2.700968  3879.532629  1503.934969  
Stressed  1.345018  3.316586  3763.602500  1556.208114

Next, the dataset is manipulated in order to keep only those features that may be useful for the intended purpose.

The next features are removed from the dataset:
- FarmID
- State
- District
- Sub-District
- HDate
- CNext
- ExpYield
- geometry
- CHeight
- evi (Problematic feature)

In [107]:
df.drop(columns=['FarmID','State','District','Sub-District','HDate','CNext','ExpYield','geometry','CHeight','evi'],inplace=True)

The SDate is manipulated to keep only the month data:

In [108]:
df['SMonth'] = df['SDate'].map(lambda x:x[-7:-5]).astype(int)
df.drop(columns='SDate',inplace=True)

The category column is converted to numerical through a fixed encoding:

In [109]:
category_encoding = {'Healthy':0,
                     'Diseased':1,
                     'Pests':2,
                     'Stressed':3}

df['y'] = df.category.map(category_encoding)
df.drop(columns=['category'],inplace=True)

The categorical variables are converted to numerical through the label-enconding technique. This is chosen for simplicity but it is important to take into account that this is better suited for ordinal variables which is not the case.

In [110]:
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()
label_encoder3 = LabelEncoder()
label_encoder4 = LabelEncoder()
label_encoder5 = LabelEncoder()
label_encoder6 = LabelEncoder()
df['crop'] = label_encoder1.fit_transform(df.Crop.values)
df['clast'] = label_encoder2.fit_transform(df.CLast.values)
df['ctransp'] = label_encoder3.fit_transform(df.CTransp.values)
df['irritype'] = label_encoder4.fit_transform(df.IrriType.values)
df['irrisource'] = label_encoder5.fit_transform(df.IrriSource.values)
df['season'] = label_encoder6.fit_transform(df.Season.values)
df.drop(columns=['Crop','CLast','CTransp', 'IrriType','IrriSource','Season'],inplace=True)

On the other hand, numerical variables are scaled as follows:

In [111]:
scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
scaler4 = StandardScaler()
scaler5 = StandardScaler()
scaler6 = StandardScaler()
scaler7 = StandardScaler()

df['crop_covered_area'] = scaler1.fit_transform(pd.DataFrame(df.CropCoveredArea))
df['water_cov'] = scaler2.fit_transform(pd.DataFrame(df.WaterCov))
df['ndvi'] = scaler3.fit_transform(pd.DataFrame(df.ndvi))
df['ndwi'] = scaler4.fit_transform(pd.DataFrame(df.ndwi))
df['gndvi'] = scaler5.fit_transform(pd.DataFrame(df.gndvi))
df['savi'] = scaler6.fit_transform(pd.DataFrame(df.savi))
df['msavi'] = scaler7.fit_transform(pd.DataFrame(df.msavi))


df.drop(columns=['CropCoveredArea','WaterCov'],inplace=True)
df.sort_index(axis='columns',inplace=True)

## Test Data Preparation

This step attemps to prepare the provided test data so that it can be evaluated by the models to be built:

In [112]:
df_test = pd.read_csv('../data/Test.csv')
#Satellite data
df_test_encoded = pd.read_csv('../data/test_encoded.csv')
df_test = pd.concat([df_test,df_test_encoded.ndvi,df_test_encoded.evi,df_test_encoded.ndwi,df_test_encoded.gndvi,
                   df_test_encoded.savi,df_test_encoded.msavi],axis=1)

Records in the test data with missing values cannot be just dropped as it was done for the training data as they are must be there so that submissions to the platform are valid.

So another approach for missing values must me performed. Missing values are fill with the variable's mean.

In [113]:
for column in [df_test.ndvi,df_test.evi,df_test.ndwi,df_test.gndvi, df_test.savi,df_test.msavi]:
    column.fillna(column.mean(),inplace=True)

Now the rest of required transformations:

In [114]:
# Not used columns
df_test.drop(columns = ['FarmID','State','District','Sub-District','HDate','CNext',
                      'ExpYield','geometry','CHeight','evi'],inplace=True)
# Date to Month
df_test['SMonth'] = df_test['SDate'].map(lambda x:x[-7:-5]).astype(int)
df_test.drop(columns = 'SDate',inplace = True)
# Label Encoding
df_test['crop'] = label_encoder1.transform(df_test.Crop.values)
df_test['clast'] = label_encoder2.transform(df_test.CLast.values)
df_test['ctransp'] = label_encoder3.transform(df_test.CTransp.values)
df_test['irritype'] = label_encoder4.transform(df_test.IrriType.values)
df_test['irrisource'] = label_encoder5.transform(df_test.IrriSource.values)
df_test['season'] = label_encoder6.transform(df_test.Season.values)
df_test.drop(columns = ['Crop','CLast','CTransp', 'IrriType','IrriSource','Season'],inplace=True)
# Scaling
df_test['crop_covered_area'] = scaler1.transform(pd.DataFrame(df_test.CropCoveredArea))
df_test['water_cov'] = scaler2.transform(pd.DataFrame(df_test.WaterCov))
df_test['ndvi'] = scaler3.fit_transform(pd.DataFrame(df_test.ndvi))
df_test['ndwi'] = scaler4.fit_transform(pd.DataFrame(df_test.ndwi))
df_test['gndvi'] = scaler5.fit_transform(pd.DataFrame(df_test.gndvi))
df_test['savi'] = scaler6.fit_transform(pd.DataFrame(df_test.savi))
df_test['msavi'] = scaler7.fit_transform(pd.DataFrame(df_test.msavi))
df_test.drop(columns = ['CropCoveredArea','WaterCov'],inplace=True)
# Sorting columns
df_test.sort_index(axis='columns',inplace=True)

## Training

The next data is used for training:

In [115]:
y_train=df.y
X_train=df.drop(columns='y')
X_test=df_test

For some cases a more balanced dataset will be used, so an undersampling of the majority class is performed:

In [116]:
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
print(y_train_resampled.value_counts())

y
0    422
1    422
2    422
3    422
Name: count, dtype: int64


### KNeighbors Algorithm

Because of the *curse of dimensionality* only three variables are used with this algorithm:

In [117]:
# Only the ndvi, savi and SMonth variables are kept
X_train_knn = pd.concat([X_train.ndvi,X_train.savi,X_train.water_cov],axis=1)


# For this case, we will divide the original training data
X_train_knn,X_test_knn,y_train_knn,y_test_knn = train_test_split(X_train_knn,y_train,test_size=0.2,
                                                               random_state=42, shuffle=True)

for n in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train_knn,y_train_knn)
    y_pred_knn = knn.predict(X_test_knn)
    f1_score_knn = f1_score(y_test_knn,y_pred_knn,average="weighted")
    print('The weighted average f1-score for',n,'neighbors is',f1_score_knn)

The weighted average f1-score for 1 neighbors is 0.69600555973801
The weighted average f1-score for 2 neighbors is 0.7326263204638507
The weighted average f1-score for 3 neighbors is 0.7378557350508214
The weighted average f1-score for 4 neighbors is 0.7315497662425016
The weighted average f1-score for 5 neighbors is 0.7304993757802747


From the last, it is possible to observe that the best number for N is 3.
(Different variable combinations were also tested despite not being shown here).

Now we train an algorithm without splitting the training data, and we test it using the actual test data:

In [118]:
X_train_knn = pd.concat([X_train.ndvi,X_train.savi,X_train.water_cov],axis=1)
X_test_knn = pd.concat([X_test.ndvi,X_test.savi,X_test.water_cov],axis=1)

knn = KNeighborsClassifier(n_neighbors=n)
knn.fit(X_train_knn,y_train)
y_pred_knn = knn.predict(X_test_knn)

### Logistic Regression

In [119]:
def compute_logistic_regression(X_train,y_train):
    model = LogisticRegression(max_iter=10000)
    model.fit(X_train,y_train)
    y_pred_training = model.predict(X_train)
    f1 = f1_score(y_train,y_pred_training,average='weighted')
    print('The weighted average f1-score for the training data is',f1)
    y_pred=model.predict(X_test)
    return y_pred

Let's try to see if it works

In [120]:
y_pred_lr=compute_logistic_regression(X_train,y_train)
print(pd.Series(y_pred_lr).value_counts())

The weighted average f1-score for the training data is 0.7456103794694815
0    3015
3       1
Name: count, dtype: int64


The algorithm doesn't work that much. It just assigns class 0 to all records. Let's try to deal with the unbalanced present in our dataset by undersampling the majority class:

In [121]:
y_pred_lr_resampled=compute_logistic_regression(X_train_resampled,y_train_resampled)
print(pd.Series(y_pred_lr_resampled).value_counts())

The weighted average f1-score for the training data is 0.31853981124505937
1    973
2    844
0    667
3    532
Name: count, dtype: int64


It didn't work neither, as now the metric dropped dramatically. The model has a very large bias.

### Decision Tree Classifier

In [ ]:
# After several tests only the ndvi, savi and water_cov variables are kept since they are giving the best results
X_train_tree = pd.concat([X_train.ndvi,X_train.savi,X_train.water_cov],axis=1)

# For this case, we will divide the original training data
X_train_tree,X_test_tree,y_train_tree,y_test_tree = train_test_split(X_train,y_train,test_size=0.2,
                                                               random_state=42, shuffle=True)
model = tree.DecisionTreeClassifier(criterion="entropy", max_depth=10)
model = model.fit(X_train_tree, y_train_tree)

y_pred_tree=model.predict(X_test_tree)

# Performance evaluation
f1_score_tree = f1_score(y_test_tree, y_pred_tree, average="weighted")  # Weighted average f1-score
print(f"The weighted average f1-score is : {f1_score_tree}")

# Classification report
print("\nClassification report :")
print(classification_report(y_test_tree, y_pred_tree))

### XGBoost

As models in general have been generating a large bias, which indicates very simple models, let's use a boosting method:


In [145]:
def create_xgboost(X_train,y_train,n,d,X_test):
    xgb_model = xgb.XGBClassifier(eval_metric='mlogloss',n_estimators=n,max_depth=d)
    xgb_model.fit(X_train, y_train)
    y_pred_training = xgb_model.predict(X_train)
    f1 = f1_score(y_train,y_pred_training,average='weighted')
    print('the weighted average f1-score in the training set is',f1)
    y_pred = xgb_model.predict(X_test)
    return y_pred

As done for KNeighbors, we do a 80-20 split in the original training data so that we can use this 20 percent as a test data where we will be able to understand the generalization capabilities of the models:

In [160]:
X_train_split,X_test_split,y_train_split,y_test_split = train_test_split(X_train,y_train,test_size=0.5,
                                                               random_state=42, shuffle=True)

for n in range(50,100,5):
    for d in range(2,6):
        print('n=',n,'d=',d,'\n')
        y_pred_split = create_xgboost(X_train_split,y_train_split,n,d,X_test_split)
        f1 = f1_score(y_test_split,y_pred_split,average='weighted')
        print('and in the test set is',f1,'\n')

n= 50 d= 2 

the weighted average f1-score in the training set is 0.7488578693674905
and in the test set is 0.7429761904761906 

n= 50 d= 3 

the weighted average f1-score in the training set is 0.7554745937123992
and in the test set is 0.742850339187808 

n= 50 d= 4 

the weighted average f1-score in the training set is 0.7841592965397438
and in the test set is 0.7424725747222608 

n= 50 d= 5 

the weighted average f1-score in the training set is 0.8467043964171623
and in the test set is 0.7421933589776176 

n= 55 d= 2 

the weighted average f1-score in the training set is 0.7488578693674905
and in the test set is 0.7429761904761906 

n= 55 d= 3 

the weighted average f1-score in the training set is 0.7572480581044051
and in the test set is 0.7427244528091455 

n= 55 d= 4 

the weighted average f1-score in the training set is 0.7903866592888201
and in the test set is 0.742450132514995 

n= 55 d= 5 

the weighted average f1-score in the training set is 0.8657132638343065
and in the tes

In general, XGBoost models show to perform similar on both splits which may be an indicator of good generalization. 

## Submissions

In [99]:
def create_submission(submission_name,y_pred):
    '''
    Creates a file ready to submit to the ZINDI platform
    '''
    submission = pd.read_csv('../data/Test.csv') # Gets the original test file which has the Farm IDs
    decoded_categories = []
    for i in y_pred:
        if i==0: decoded_categories.append('Healthy')
        elif i==1: decoded_categories.append('Diseased')
        elif i==2: decoded_categories.append('Pests')
        elif i==3: decoded_categories.append('Stressed')
    submission = pd.concat([submission.FarmID,pd.Series(decoded_categories)],axis=1)
    submission.columns = ['ID','Target']
    submission.to_csv('../data/submissions/submission'+submission_name+'.csv',index=False)

**Submission #J2:**
Logistic regressor built with all the features present in X_train.

In [100]:
#y_pred_lr=compute_logistic_regression(X_train,y_train)
#create_submission('j2',y_pred_lr_resampled)

**Submission #J3**: XGBoost (By default 100 estimators and a depth of 6)

In [101]:
y_pred_j3=create_xgboost(X_train,y_train,X_test)
create_submission('j3',y_pred_j3)

the weighted average f1-score in the training set is 0.9439670747773803


**Submission #J4**: XGBoost, with 3 estimators and a depth of 4

In [151]:
y_pred_j4=create_xgboost(X_train,y_train,3,4,X_test)
create_submission('j4',y_pred_j4)

the weighted average f1-score in the training set is 0.746523058728019


**Submission #J5**: XGBoost, with 100 estimators and a depth of 5

In [161]:
y_pred_j5=create_xgboost(X_train,y_train,100,5,X_test)
print(pd.Series(y_pred_j5).value_counts())
create_submission('j5',y_pred_j5)

the weighted average f1-score in the training set is 0.8685967027144323
0    2999
2      16
3       1
Name: count, dtype: int64


**Submission #J6**: XGBoost, with 50 estimators and a depth of 5

In [162]:
y_pred_j6=create_xgboost(X_train,y_train,50,5,X_test)
print(pd.Series(y_pred_j6).value_counts())
create_submission('j6',y_pred_j6)

the weighted average f1-score in the training set is 0.78413519383832
0    3006
2       9
1       1
Name: count, dtype: int64


**Submission #J7**: XGBoost, with 100 estimators and a depth of 4

In [163]:
y_pred_j7=create_xgboost(X_train,y_train,100,4,X_test)
print(pd.Series(y_pred_j7).value_counts())
create_submission('j7',y_pred_j7)

the weighted average f1-score in the training set is 0.7926450864304826
0    3003
2       9
3       3
1       1
Name: count, dtype: int64


**Submission #J8**: XGBoost, with 100 estimators and a depth of 3

In [164]:
y_pred_j8=create_xgboost(X_train,y_train,100,3,X_test)
print(pd.Series(y_pred_j8).value_counts())
create_submission('j8',y_pred_j8)

the weighted average f1-score in the training set is 0.7561333196455832
0    2995
2      17
1       3
3       1
Name: count, dtype: int64


**Submission #J9**: XGBoost, with 75 estimators and a depth of 4

In [165]:
y_pred_j9=create_xgboost(X_train,y_train,75,4,X_test)
print(pd.Series(y_pred_j9).value_counts())
create_submission('j9',y_pred_j9)

the weighted average f1-score in the training set is 0.773659822526772
0    3004
2       8
3       3
1       1
Name: count, dtype: int64


**Submission #J10**: XGBoost, with 150 estimators and a depth of 4

In [166]:
y_pred_j10=create_xgboost(X_train,y_train,150,4,X_test)
print(pd.Series(y_pred_j10).value_counts())
create_submission('j10',y_pred_j10)

the weighted average f1-score in the training set is 0.8334713610492466
0    2993
2      16
3       4
1       3
Name: count, dtype: int64


In [167]:
y_pred_j11=create_xgboost(X_train,y_train,125,4,X_test)
print(pd.Series(y_pred_j11).value_counts())
create_submission('j11',y_pred_j11)

the weighted average f1-score in the training set is 0.8106949789359852
0    2996
2      16
3       3
1       1
Name: count, dtype: int64
